In [95]:
import numpy as np
from utils import Load
from tensorflow import keras

## Carrega os dados

In [96]:
data = Load("train_sequences.csv", "train_labels.csv")

In [97]:
data

,target_id,molecule,ID,resname,resid,x_1,y_1,z_1
0,1SCL_A,1,1SCL_A_1,G,1.0,13.760000,-25.974001,0.102000
1,1SCL_A,1,1SCL_A_2,G,2.0,9.310000,-29.638000,2.669000
2,1SCL_A,1,1SCL_A_3,G,3.0,5.529000,-27.813000,5.878000
3,1SCL_A,4,1SCL_A_4,U,4.0,2.678000,-24.900999,9.793000
4,1SCL_A,1,1SCL_A_5,G,5.0,1.827000,-20.136000,11.793000
...,...,...,...,...,...,...,...,...
133208,8T2Y_tP,1,8T2Y_tP_17,G,17.0,183.376007,231.067001,257.790985
133209,8T2Y_tP,1,8T2Y_tP_18,G,18.0,179.199997,229.108994,262.325012
133210,8T2Y_tP,3,8T2Y_tP_19,A,19.0,182.572998,222.063004,264.239014
133211,8T2Y_tP,3,8T2Y_tP_20,A,20.0,181.662003,217.369995,258.062012


## Separa 80 sequencias para validacao

Devido ao arquivo de validacao conter apenas 12 sequencias

In [98]:
train_data = data[data["target_id"].isin(data["target_id"].unique()[:-80])]
valid_data = data[data["target_id"].isin(data["target_id"].unique()[-80:])]

print(train_data.shape)
print(valid_data.shape)

(8316, 8)
(1680, 8)


## Separa os dados em treino e validacao

In [99]:
X_train = train_data.groupby("target_id").apply(lambda group: group[:-1])[["x_1", "y_1", "z_1", "molecule", "resid"]]
y_train = train_data.groupby("target_id").last()

X_valid = valid_data.groupby("target_id").apply(lambda group: group[:-1])[["x_1", "y_1", "z_1", "molecule", "resid"]]
y_valid = valid_data.groupby("target_id").last()

/tmp/ipykernel_52950/618353232.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  X_train = train_data.groupby("target_id").apply(lambda group: group[:-1])[["x_1", "y_1", "z_1", "molecule", "resid"]]
/tmp/ipykernel_52950/618353232.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  X_valid = valid_data.groupby("target_id").apply(lambda group: group[:-1])[["x_1", "y_1", "z_1", "molecule", "resid"]]


In [100]:
y_train = y_train[["x_1", "y_1", "z_1"]].astype(np.float32)
y_valid = y_valid[["x_1", "y_1", "z_1"]].astype(np.float32)

In [101]:
X_train = X_train.values.reshape(len(train_data["target_id"].unique()), 20, 5).astype(np.float32)
X_valid = X_valid.values.reshape(len(valid_data["target_id"].unique()), 20, 5).astype(np.float32)

## Constroi o modelo com RNN

In [102]:
model = keras.Sequential([
    keras.layers.Input([None, 5]),
    keras.layers.SimpleRNN(20, return_sequences=True, kernel_regularizer="l2"),
    keras.layers.SimpleRNN(20, kernel_regularizer="l2"),
    keras.layers.Dense(3, activation="linear", kernel_regularizer="l2")
])

In [103]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_8 (SimpleRNN)        │ (None, None, 20)       │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_9 (SimpleRNN)        │ (None, 20)             │           820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,403 (5.48 KB)

 Trainable params: 1,403 (5.48 KB)

 Non-trainable params: 0 (0.00 B)

In [104]:
model.compile(loss="mse", optimizer="adam")

In [105]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=128, validation_data=(X_valid, y_valid))

Epoch 1/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - loss: 9668.8145 - val_loss: 22513.2441
Epoch 2/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 8526.3809 - val_loss: 22444.9883
Epoch 3/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 8487.2891 - val_loss: 22387.2520
Epoch 4/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 8741.1094 - val_loss: 22329.7207
Epoch 5/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 8417.6074 - val_loss: 22270.2441
Epoch 6/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 9409.2324 - val_loss: 22210.9941
Epoch 7/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 8177.1597 - val_loss: 22155.7500
Epoch 8/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 8446.1748 - val_loss: 22107.0703
Epoch 9/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 8841.7725 - val_loss: 22061.7695
Epoch 10/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 9766.3242 - val_loss: 22019.8594
Epoch 11/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 9134.0605 - val_los